### Check for transferts with ambiguous invoice dates


In [ ]:
import pandas as pd
from pathlib import Path
from datetime import date


file_path = (
    Path.home()
    / "Nextcloud/TSOSI_data/doaj/3_enriched/2025-01-15-DOAJ_Library_Report_2024_enriched_done.xlsx"
)

df = pd.read_excel(file_path)

df["_invoice_date"] = pd.to_datetime(df["Invoice date"], errors="coerce")

date_bound = date(2023, 7, 1)
df["_date_before_bound"] = df["_invoice_date"].apply(
    lambda x: True if pd.isna(x) else x.date() < date_bound
)


mask = (
    ~df["Support amount"].isna()
    & ~df["_invoice_date"].isna()
    & df["_date_before_bound"]
)
columns = [c for c in df.columns if not c[0] == "_"]
to_review = df[mask][columns]
to_review.to_excel(
    "TSOSI_DOAJ_Library_2024_Invoice_data_review.xlsx", index=False
)

### Check for duplicated transferts


In [19]:
import pandas as pd
from pathlib import Path
from datetime import date


file_path = (
    Path.home()
    / "Nextcloud/TSOSI_data/doaj/3_enriched/2024-09-20-DOAJ_Library_Report_2021_enriched_done.xlsx"
)

amount_field = "amount"
entity_field = "Institution name"

df = pd.read_excel(file_path)

columns = [c for c in df.columns if not c[0] == "_"]
# Drop lines with missing amounts - They correspond to past transferts.
df = df[~df[amount_field].isna()][columns].copy()
# Check for entities with multiple occurences
grouped = df.groupby(entity_field)[amount_field].count()
grouped[grouped > 1]
mask = df[entity_field].isin(grouped[grouped > 1].index)
to_review = df[mask].sort_values(entity_field)
# Export
# to_review.to_excel(
#     "TSOSI_DOAJ_Library_2024_Duplicated_transferts_review.xlsx", index=False
# )
to_review

,Institution name,type,country,currency,amount
149,Université de Bordeaux,Library,France,€,1317.72
150,Université de Bordeaux,Library,France,€,1931.70
